In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [2]:
data_url = 'http://lib.stat.cmu.edu/datasets/boston'

raw_data = pd.read_csv(data_url, sep='\s+',skiprows=22, header=None)

data = np.hstack([raw_data.values[::2,:],raw_data.values[1::2,:2]])

target = raw_data.values[1::2,2]

x = data
y = target


In [3]:
raw_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10
0,0.00632,18.00,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3
1,396.90000,4.98,24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.02731,0.00,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8
3,396.90000,9.14,21.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.02729,0.00,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8
